In [1]:
import pyaudio
import os
import struct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

from keras.models import Sequential, Model, model_from_json
from keras import losses
import keras 
import pickle
import wave  # !pip install wave
# import os
import sys
import warnings
import librosa
import librosa.display
import IPython.display as ipd  # To play sound in the notebook
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers


In [2]:
# # loading json and model architecture :
# print("--> model architecture ..")
# json_file = open('./model/model_json_1D.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

# # load weights into new model
# print("--> Loading weights ..")
# loaded_model.load_weights("./model/SER_Model_1D.h5")
# print("Model loaded successfully !")

# # the optimiser
# print("Optimization ..")
# opt = optimizers.RMSprop(lr=0.00001, decay=1e-6)
# loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 5))
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)
# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)
# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)
# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
# ax1.set_xlabel('samples')
# ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])
# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    # binary data
    data = stream.read(CHUNK)  
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    # # Apply predictions
    # print("Apply Predictions ..")
    # data_2 = np.frombuffer(data, dtype=np.float32)
    # mfccs = np.mean(librosa.feature.mfcc(y=data_2, sr=RATE, n_mfcc=13),axis=0)
    # newdf = pd.DataFrame(data=mfccs).T
    # newdf= np.expand_dims(newdf, axis=2)
    # newpred = loaded_model.predict(newdf, batch_size=16, verbose=1)
    # print("Append labels ..")
    # infile = open('./model/labels','rb')
    # lb = pickle.load(infile)
    # infile.close()
    # print("Getting Result ..")
    # result = newpred.argmax(axis=1)
    # result = result.astype(int).flatten()
    # result = (lb.inverse_transform((result)))
    # print(result)
    
    line.set_ydata(data_np)
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1 
    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break
        
        
# stop stream
stream.stop_stream()
stream.close()
# close PyAudio
p.terminate()



stream started
